In [1]:
# Select TensorFlow 2.0
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
# Install Weights and Biases
!pip install wandb

In [0]:
# Imports
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from tensorflow.keras import utils
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import TensorBoard

import wandb
from wandb.keras import WandbCallback

In [0]:
# Login to your W&B account
!wandb login

In [5]:
# Initialize your W&B project allowing it to sync with TensorBoard
wandb.init(project="tensorboard-integration-partII", sync_tensorboard=True)
config = wandb.config

In [0]:
# Specify the configuration variables
config.dropout = 0.2
config.hidden_layer_size = 128
config.layer_1_size  = 16
config.layer_2_size = 32
config.learn_rate = 0.01
config.decay = 1e-6
config.momentum = 0.9
config.epochs = 25

In [7]:
# Load and preprocess the data
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
labels=["T-shirt/top","Trouser","Pullover","Dress","Coat",
        "Sandal","Shirt","Sneaker","Bag","Ankle boot"]

img_width=28
img_height=28

X_train = X_train.astype('float32')
X_train /= 255.
X_test = X_test.astype('float32')
X_test /= 255.

# Reshape input data
X_train = X_train.reshape(X_train.shape[0], img_width, img_height, 1)
X_test = X_test.reshape(X_test.shape[0], img_width, img_height, 1)

# One hot encode outputs
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)
num_classes = y_test.shape[1]

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
# Train the model
sgd = SGD(lr=config.learn_rate, decay=config.decay, momentum=config.momentum,
                            nesterov=True)

model = Sequential()
model.add(Conv2D(config.layer_1_size, (5, 5), activation='relu',
                            input_shape=(img_width, img_height,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(config.layer_2_size, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(config.dropout))
model.add(Flatten())
model.add(Dense(config.hidden_layer_size, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# The WandbCallback logs metrics and some examples of the test data
model.fit(X_train, y_train,  validation_data=(X_test, y_test), epochs=config.epochs,
    callbacks=[WandbCallback(data_type="image", labels=labels), 
               TensorBoard(log_dir=wandb.run.dir)])

Train on 60000 samples, validate on 10000 samples
Epoch 1/25
60000/60000 [==============================] - 16s 260us/sample - loss: 0.5526 - accuracy: 0.7980 - val_loss: 0.3900 - val_accuracy: 0.8572
Epoch 2/25
60000/60000 [==============================] - 8s 138us/sample - loss: 0.3708 - accuracy: 0.8651 - val_loss: 0.3398 - val_accuracy: 0.8751
Epoch 3/25
60000/60000 [==============================] - 8s 134us/sample - loss: 0.3329 - accuracy: 0.8784 - val_loss: 0.3359 - val_accuracy: 0.8722
Epoch 4/25
60000/60000 [==============================] - 8s 132us/sample - loss: 0.3091 - accuracy: 0.8854 - val_loss: 0.3047 - val_accuracy: 0.8898
Epoch 5/25
60000/60000 [==============================] - 8s 132us/sample - loss: 0.2913 - accuracy: 0.8921 - val_loss: 0.3076 - val_accuracy: 0.8889
Epoch 6/25
60000/60000 [==============================] - 8s 134us/sample - loss: 0.2769 - accuracy: 0.8971 - val_loss: 0.2917 - val_accuracy: 0.8911
Epoch 7/25
60000/60000 [=========================

Run page: https://app.wandb.ai/sayakpaul/tensorboard-integration-partII

## Plotting confusion matrix in TensorBoard

Reference: https://www.tensorflow.org/tensorboard/image_summaries

In [0]:
from sklearn.metrics import confusion_matrix
from datetime import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import itertools
import io

In [47]:
# Re-init wandb
wandb.init(project="tensorboard-integration-partII", sync_tensorboard=True)

W&B Run: https://app.wandb.ai/sayakpaul/tensorboard-integration-partII/runs/weahshb2

In [0]:
# Configure the run directory
logdir = wandb.run.dir
file_writer_cm = tf.summary.create_file_writer(logdir)

In [0]:
def plot_confusion_matrix(cm, class_names):
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    return figure

In [0]:
def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image

In [0]:
def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset
    test_pred_raw = model.predict(X_test)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix
    cm = confusion_matrix(np.argmax(y_test, axis=1), test_pred)
    # Log the confusion matrix as an image summary
    figure = plot_confusion_matrix(cm, class_names=labels)
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [0]:
# Define the per-epoch callback
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [53]:
# Kickstart the model training process
sgd = SGD(lr=config.learn_rate, decay=config.decay, momentum=config.momentum,
                            nesterov=True)

model = Sequential()
model.add(Conv2D(config.layer_1_size, (5, 5), activation='relu',
                            input_shape=(img_width, img_height,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(config.layer_2_size, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(config.dropout))
model.add(Flatten())
model.add(Dense(config.hidden_layer_size, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# The WandbCallback logs metrics and some examples of the test data
model.fit(X_train, y_train,  validation_data=(X_test, y_test), epochs=config.epochs,
    callbacks=[WandbCallback(data_type="image", labels=labels), 
               TensorBoard(log_dir=logdir),
               cm_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/25
60000/60000 [==============================] - 10s 174us/sample - loss: 0.5728 - accuracy: 0.7892 - val_loss: 0.4258 - val_accuracy: 0.8434
Epoch 2/25
60000/60000 [==============================] - 10s 164us/sample - loss: 0.3899 - accuracy: 0.8583 - val_loss: 0.3550 - val_accuracy: 0.8677
Epoch 3/25
60000/60000 [==============================] - 10s 162us/sample - loss: 0.3415 - accuracy: 0.8748 - val_loss: 0.3360 - val_accuracy: 0.8779
Epoch 4/25
60000/60000 [==============================] - 10s 161us/sample - loss: 0.3155 - accuracy: 0.8830 - val_loss: 0.3022 - val_accuracy: 0.8885
Epoch 5/25
60000/60000 [==============================] - 10s 162us/sample - loss: 0.2973 - accuracy: 0.8886 - val_loss: 0.3034 - val_accuracy: 0.8865
Epoch 6/25
60000/60000 [==============================] - 10s 166us/sample - loss: 0.2833 - accuracy: 0.8947 - val_loss: 0.2888 - val_accuracy: 0.8938
Epoch 7/25
60000/60000 [====================